##### ### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2025 Semester 1

## Assignment 1: Scam detection with naive Bayes


**Student ID(s):**     `1409447`


This iPython notebook is a template which you will use for your Assignment 1 submission.

**NOTE: YOU SHOULD ADD YOUR RESULTS, GRAPHS, AND FIGURES FROM YOUR OBSERVATIONS IN THIS FILE TO YOUR REPORT (the PDF file).** Results, figures, etc. which appear in this file but are NOT included in your report will not be marked.

**Adding proper comments to your code is MANDATORY. **

## 1. Supervised model training


In [16]:
# Imports
import pandas as pd


# Load data from sms_supervised_train.csv
supervised_df = pd.read_csv('COMP30027_2025_asst1_data/sms_supervised_train.csv')
supervised_df.head()

# First, define the vocabulary -- create a dictionary for [word: count]
word_counts = {}

for text in supervised_df["textPreprocessed"]:
    # If text is empty, ignore it as it doesn't provide any learning data
    if text == "":
        print("found one")
    # Split text using space as delim
    text = str(text)
    words = text.split(" ")
    for word in words:
        if word not in word_counts.keys():
            word_counts[word] = 0
        word_counts[word] += 1

print(word_counts)
sorted_keys = list(word_counts.keys()).sort()
for i in sorted_keys:
    print(i)
    

{'?': 533, 'u': 396, 'workin': 2, '.': 1935, '..': 278, 'sir': 16, ',': 679, 'send': 102, 'email': 14, 'log': 10, 'payment': 4, 'll': 12, 'another': 15, 'message': 52, 'explain': 2, 'thing': 37, 'back': 59, 'home': 57, 'great': 51, 'weekend': 17, '2': 174, '4': 122, 'quite': 14, 'get': 222, 'ok': 96, 'lor': 55, 'anyway': 7, 'thk': 15, 'ticket': 10, 'co': 15, 'like': 86, 'late': 75, 'already': 28, 'wan': 17, 'go': 215, 'look': 34, 'ur': 151, 'frens': 5, 'darren': 7, 'wif': 11, '...': 336, 'wat': 33, 'r': 57, 'reason': 6, 'speak': 17, 'year': 34, 'anyways': 1, 'week': 62, 'good': 99, 'exam': 4, 'stop': 76, 'story': 6, 'tell': 69, 'return': 7, 'he': 6, "'s": 108, 'say': 32, 're': 13, 'order': 12, '(': 47, 'text': 93, 'time': 72, ')': 58, 'sch': 7, 'neva': 7, 'mind': 14, 'eat': 21, '1st': 20, 'reply': 75, '21': 3, '/': 93, 'f': 3, 'invite': 9, 'friend': 31, ':': 109, 'frnd': 5, '62468': 3, 'o': 4, 'turn': 1, 'love': 93, 'phone': 59, 'online': 14, 'transaction': 4, 'da': 38, 'ü': 61, 'noe':

TypeError: 'NoneType' object is not iterable

## 2. Supervised model evaluation

## 3. Extending the model with semi-supervised training

## 4. Supervised model evaluation